### python과 mysql을 연동 
1. 외부의 라이브러리 설치 (pymysql)
2. 라이브러리 로드 
3. sql 서버의 정보를 등록하여 접속 
4. sql 쿼리문을 작성하여 sql 서버에 쿼리문을 요청 
5. 쿼리문에 따라서 응답 메시지 출력

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [65]:
# 라이브러리 로드 
import pymysql

In [66]:
# sql 서버의 정보를 입력 -> 접속
# 내 컴퓨터에 연결 
_db = pymysql.connect(
    host = '127.0.0.1', 
    port = 3306, 
    user = 'root', 
    password = '1234', 
    db = 'ezen'
)

In [ ]:
## Cursor 생성 
# DictCursor로 cursor 생성 
cursor = _db.cursor(pymysql.cursors.DictCursor)
# 기본 값 사용
cursor2 = _db.cursor()

In [ ]:
# sql 쿼리문을 작성 
sql = """
    select 
    * 
    from 
    emp
"""

In [ ]:
# 쿼리문을 DB server에 요청 
cursor.execute(sql)

In [ ]:
data = cursor.fetchall()
data

In [ ]:
cursor2.execute(sql)

In [ ]:
data2 = cursor2.fetchall()
data2

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(data)

In [ ]:
pd.DataFrame(data2)

In [ ]:
# 데이터를 추가 
sql = """
    insert into 
    emp 
    values 
    (1111, 'test', 'SALESMAN', 1111, '1980-01-01', 1000, 0, 10)
"""

In [ ]:
cursor.execute(sql)

In [ ]:
cursor.fetchall()

In [ ]:
select_sql = """
    select * from emp
"""

In [ ]:
cursor.execute(select_sql)
data = cursor.fetchall()
pd.DataFrame(data)

In [ ]:
# cursor의 데이터를 db server에 동기화 
_db.commit()

- sql 쿼리문이 select인 경우
    - cursor.execute()
    - cursor.fetchall()
- sql 쿼리문이 select가 아닌 경우 
    - cursor.execute()
    - _db.commit()

In [ ]:
select_query = """
    select * from emp
"""
update_query = """
    update emp_selector set SAL = 1100 where ENAME='test'
"""

In [ ]:
print(select_sql.find('select'))
print(update_query.find('select'))

In [ ]:
print('select' in select_query)
print('select' in update_query)

In [ ]:
select_query.strip().startswith('select')

In [ ]:
update_query.split()[0] == 'select'

In [ ]:
delete_query = """
    delete from emp where ENAME = %s
"""
input_ename = input('삭제할 사원의 이름을 입력하시오 : ')

cursor.execute(delete_query, [input_ename])

In [ ]:
_db.commit()

### query를 실행하는 함수를 생성 
- 매개변수 2개를 사용 ( query, *data )
- query 매개변수가 select로 시작한다면
- cursor.execute(query, data) -> cursor.fetchall() -> 데이터프레임을 생성하여 리턴
- select로 시작하지 않는다면 
- cursor.execute(query, data) -> _db.commit() -> "Query OK" 문자열을 리턴

In [ ]:
def sql_query( query, *data ):
    cursor.execute(query, data)
    # query의 시작이 select라면?
    if query.upper().split()[0] == 'SELECT':
        data = cursor.fetchall()
        result = pd.DataFrame(data)
    else:
        _db.commit()
        result = "Query OK"
    return result

In [ ]:
# emp 테이블에서 SAL가 2000 이상한 사원의 정보를 확인 
select1 = """
    select * from emp where SAL >= 2000
"""
sql_query(select1)

In [ ]:
# emp에서 SAL이 1500 이상이고 3000 이하인 사원의 정보를 출력 
# case1
select2 = """
    select * from emp where SAL >= 1500 and SAL <= 3000
"""
sql_query(select2)

In [ ]:
select3 = """
    select * from emp where SAL between 1500 and 3000
"""
sql_query(select3)

In [ ]:
## JOB이 'SALESMAN'이거나 'MANAGER' 인 사원 정보를 출력 
# case1
select4 = """
    select * from emp where JOB = 'SALESMAN' OR JOB = 'MANAGER'
"""
sql_query(select4)

In [ ]:
# case2
select5 = """
    select * from emp where JOB IN ('SALESMAN', 'MANAGER')
"""
sql_query(select5)

In [ ]:
# 사원의 이름이 S로 시작하는 사원 정보를 출력 
select6 = """
    select * from emp where ENAME LIKE "S%%"
"""
sql_query(select6)

In [ ]:
# 그룹화 
# JOB을 기준으로 그룹화하여 SAL 평균을 구한다. 
select7 = """
    select JOB as 직무, AVG(SAL) as 평균연봉 from emp group by JOB
"""
sql_query(select7)

In [ ]:
## table 결합 
## join 결합 (emp, dept)
join1 = """
    select * from emp left join dept on emp.deptno = dept.deptno
"""
sql_query(join1)

In [ ]:
## 부서의 지역이 NEW YORK인 사원의 정보를 출력 
# join 
join2 = """
    select * from emp left join dept 
    on emp.deptno = dept.deptno where LOC = 'NEW YORK'
"""
sql_query(join2)

In [ ]:
select8 = """
    select deptno from dept where LOC != 'NEW YORK'
"""
sql_query(select8)

In [ ]:
# 서브쿼리
sub1 = """
    select * from emp where deptno in (
        select deptno from dept where LOC != 'NEW YORK'
    )
"""
sql_query(sub1)

In [ ]:
import mod_sql as ms

In [ ]:
# ms 안에 있는 Mydb class 생성 
# 기본값으로 class 생성 
db1 = ms.Mydb()

In [ ]:
db2 = ms.Mydb(
    "moons-86.iptime.org", 
    3306, 
    'ubion', 
    '1234', 
    'ubion'
)

In [ ]:
sql = """
    select * from corona
"""

In [ ]:
db1.sql_query(sql)

In [64]:
data = db2.sql_query(sql)

In [ ]:
### sql 유니언 결합
# tran_1 table과 tran_2 table을 유니언 결합 
union1 = """
    select * from tran_1
    union 
    select * from tran_2
"""
db2.sql_query(union1)

In [72]:
union2 = """
    select * from 
    (select * from tran_1
    union 
    select * from tran_2) as tran 
    left join 
    (select * from tran_d_1
    union 
    select * from tran_d_2) as tran_d
    on tran.transaction_id = tran_d.transaction_id
"""
db2.sql_query(union2)

,transaction_id,price,payment_date,customer_id,detail_id,tran_d.transaction_id,item_id,quantity
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,T0000000113,S005,1
1,T0000000114,50000,2019-02-01 01:37:23,HD678019,1,T0000000114,S001,1
2,T0000000115,120000,2019-02-01 02:34:19,HD298120,2,T0000000115,S003,1
3,T0000000116,210000,2019-02-01 02:47:23,IK452215,3,T0000000116,S005,1
4,T0000000117,170000,2019-02-01 04:33:46,PL542865,4,T0000000117,S002,2
...,...,...,...,...,...,...,...,...
7139,T0000006887,210000,2019-07-31 19:43:42,AS961301,7132,T0000006887,S005,1
7140,T0000006888,210000,2019-07-31 20:05:45,GD025393,7133,T0000006888,S005,1
7141,T0000006891,50000,2019-07-31 20:28:20,HI337854,7136,T0000006891,S001,1
7142,T0000006894,180000,2019-07-31 21:20:44,HI400734,7139,T0000006894,S004,1
